# Import raw data files

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import glob

In [ ]:
#path=r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\aclImdb"


In [5]:
#read train pos files
text=[]
file_names=[]
is_positive=[]
#path=r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\aclImdb\train\neg"
#filenames=glob.glob(path + "/*.txt")
with os.scandir(r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\Group project 1\aclImdb\train\pos") as filenames:
    for file in filenames:
        head, tail = os.path.split(file)
        file_names.append(tail)
        is_positive.append(1)
        with open(file, 'rb') as open_file:
            text.append(open_file.readlines()[0])

In [6]:
train_df=pd.DataFrame({'file':file_names, 'text':text, 'is_positive':is_positive})

In [ ]:
train_df

In [7]:
#read train neg files
with os.scandir(r"C:\Users\yunan\Downloads\York U\Machine Learning Cert\ML1010\Group project 1\aclImdb\train\neg") as filenames:
    for file in filenames:
        head,tail=os.path.split(file)
        file_names.append(tail)
        is_positive.append(0)
        with open(file,'rb') as open_file:
            text.append(open_file.readlines()[0])

In [8]:
train_df=pd.DataFrame({'file':file_names, 'text':text, 'is_positive':is_positive})

In [9]:
train_df['text']=train_df['text'].astype(str)

In [25]:
# reduce dataset size - select only 5000 records
positive_reviews = train_df[:2500]   #take first 2500 reviews which are positive
negative_reviews = train_df[22500:]  #last last 2500 reviews which are negative
new_train = positive_reviews.append(negative_reviews, ignore_index=True)

In [26]:
new_train['text']=new_train['text'].astype(str)
new_train.head(5)

,file,text,is_positive
0,0_9.txt,b'Bromwell High is a cartoon comedy. It ran at...,1
1,10000_8.txt,b'Homelessness (or Houselessness as George Car...,1
2,10001_10.txt,b'Brilliant over-acting by Lesley Ann Warren. ...,1
3,10002_7.txt,b'This is easily the most underrated film inn ...,1
4,10003_8.txt,b'This is not the typical Mel Brooks film. It ...,1


#  Text Pre-processing

In [12]:
import re
import nltk
import matplotlib.pyplot as plt
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yunan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from tokenize import tokenize

In [14]:
wpt=nltk.WordPunctTokenizer()
stop_words=nltk.corpus.stopwords.words('english')

In [15]:
from nltk.tokenize.casual import casual_tokenize

In [16]:
from nltk.stem.snowball import SnowballStemmer #  Snowball stemmer > Porter stemmer
from nltk.tokenize.casual import casual_tokenize  # we use casual tokenize because this is colloquial text
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words  # sklearn stop words is larger than nltk

In [19]:
stemmer = SnowballStemmer("english")
def decontracted(phrase):
    # Taken from https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
    
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def tokenize_phrase(text):
    phrase = decontracted(text.replace('\\', '').replace("b'", "")) # remove backslashes and replace contractions
    tokens = casual_tokenize(phrase, reduce_len=True, strip_handles=True)
    normalized_tokens = [x.lower() for x in tokens] #  convert to all lowercase
    filtered_tokens = [x for x in normalized_tokens if x not in sklearn_stop_words] #  filter stop words
    filtered_tokens = [x for x in filtered_tokens if x and x not in '- \t\n."\':[...][\\]()/[br]<>*~,;!?'] #  filter punctuations
    stemmed_tokens = [stemmer.stem(w) for w in filtered_tokens] # perform stemming
    return stemmed_tokens


In [20]:
new_train['tokens'] = new_train['text'].apply(tokenize_phrase)
new_train.head(5)

,file,text,is_positive,tokens
0,0_9.txt,b'Bromwell High is a cartoon comedy. It ran at...,1,"[bromwel, high, cartoon, comedi, ran, time, pr..."
1,10000_8.txt,b'Homelessness (or Houselessness as George Car...,1,"[homeless, houseless, georg, carlin, state, is..."
2,10001_10.txt,b'Brilliant over-acting by Lesley Ann Warren. ...,1,"[brilliant, over-act, lesley, ann, warren, bes..."
3,10002_7.txt,b'This is easily the most underrated film inn ...,1,"[easili, underr, film, inn, brook, cannon, sur..."
4,10003_8.txt,b'This is not the typical Mel Brooks film. It ...,1,"[typic, mel, brook, film, slapstick, movi, act..."


In [21]:
import spacy

In [22]:
nlp=spacy.load("en_core_web_sm")

In [35]:
doc=nlp(str(new_train['text']))

In [36]:
doc

0       b'Bromwell High is a cartoon comedy. It ran at...
1       b'Homelessness (or Houselessness as George Car...
2       b'Brilliant over-acting by Lesley Ann Warren. ...
3       b'This is easily the most underrated film inn ...
4       b'This is not the typical Mel Brooks film. It ...
                              ...                        
4995    b"Towards the end of the movie, I felt it was ...
4996    b'This is the kind of movie that my enemies co...
4997    b"I saw 'Descent' last night at the Stockholm ...
4998    b"Some films that you pick up for a pound turn...
4999    b"This is one of the dumbest films, I've ever ...
Name: text, Length: 5000, dtype: object

In [37]:
token_list=[token for token in doc]

In [43]:
token_list

[0,
       ,
 b'Bromwell,
 High,
 is,
 a,
 cartoon,
 comedy,
 .,
 It,
 ran,
 at,
 ...,
 ,
 1,
       ,
 b'Homelessness,
 (,
 or,
 Houselessness,
 as,
 George,
 Car,
 ...,
 ,
 2,
       ,
 b'Brilliant,
 over,
 -,
 acting,
 by,
 Lesley,
 Ann,
 Warren,
 .,
 ...,
 ,
 3,
       ,
 b'This,
 is,
 easily,
 the,
 most,
 underrated,
 film,
 inn,
 ...,
 ,
 4,
       ,
 b'This,
 is,
 not,
 the,
 typical,
 Mel,
 Brooks,
 film,
 .,
 It,
 ...,
 
                               ,
 ...,
                        ,
 4995,
    ,
 b"Towards,
 the,
 end,
 of,
 the,
 movie,
 ,,
 I,
 felt,
 it,
 was,
 ...,
 ,
 4996,
    ,
 b'This,
 is,
 the,
 kind,
 of,
 movie,
 that,
 my,
 enemies,
 co,
 ...,
 ,
 4997,
    ,
 b"I,
 saw,
 ',
 Descent,
 ',
 last,
 night,
 at,
 the,
 Stockholm,
 ...,
 ,
 4998,
    ,
 b"Some,
 films,
 that,
 you,
 pick,
 up,
 for,
 a,
 pound,
 turn,
 ...,
 ,
 4999,
    ,
 b"This,
 is,
 one,
 of,
 the,
 dumbest,
 films,
 ,,
 I,
 've,
 ever,
 ...,
 ,
 Name,
 :,
 text,
 ,,
 Length,
 :,
 5000,
 ,,
 dt

In [41]:
filtered_tokens=[token for token in token_list if not token.is_stop]

In [42]:
filtered_tokens

[0,
       ,
 b'Bromwell,
 High,
 cartoon,
 comedy,
 .,
 ran,
 ...,
 ,
 1,
       ,
 b'Homelessness,
 (,
 Houselessness,
 George,
 Car,
 ...,
 ,
 2,
       ,
 b'Brilliant,
 -,
 acting,
 Lesley,
 Ann,
 Warren,
 .,
 ...,
 ,
 3,
       ,
 b'This,
 easily,
 underrated,
 film,
 inn,
 ...,
 ,
 4,
       ,
 b'This,
 typical,
 Mel,
 Brooks,
 film,
 .,
 ...,
 
                               ,
 ...,
                        ,
 4995,
    ,
 b"Towards,
 end,
 movie,
 ,,
 felt,
 ...,
 ,
 4996,
    ,
 b'This,
 kind,
 movie,
 enemies,
 co,
 ...,
 ,
 4997,
    ,
 b"I,
 saw,
 ',
 Descent,
 ',
 night,
 Stockholm,
 ...,
 ,
 4998,
    ,
 b"Some,
 films,
 pick,
 pound,
 turn,
 ...,
 ,
 4999,
    ,
 b"This,
 dumbest,
 films,
 ,,
 ...,
 ,
 :,
 text,
 ,,
 Length,
 :,
 5000,
 ,,
 dtype,
 :,
 object]

In [ ]:
def normalize_document(doc):
    # lower

In [ ]:
def get_data(split):
    if split.lower() == 'train':
        folder = 'train'
    elif split.lower() == 'test':
        folder = 'test'
    else:
        raise ValueError('Invalid data split specified.')
        
    file_names = []
    text = []
    is_positive = []
    
    # read all positive files
    files = glob.glob(os.path.join("data", folder, 'pos', '*'))
    for file in files:
        head, tail = os.path.split(file)
        file_names.append(tail)
        is_positive.append(1)
        with open(file, 'rb') as open_file:
            text.append(open_file.readlines()[0])
            
    # read all negative files
    files = glob.glob(os.path.join("data", folder, 'neg', '*'))
    for file in files:
        head, tail = os.path.split(file)
        file_names.append(tail)
        is_positive.append(0)
        with open(file, 'rb') as open_file:
            text.append(open_file.readlines()[0])
            
    return pd.DataFrame(data={'file': file_names, 'text': text, 'is_positive': is_positive})


train_df = get_data('train')
train_df['text'] = train_df['text'].astype(str)
train_df